<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🟣 תהליך יצירת נקודות גבול בתהליך לבנק"ל מודרני 🟣
    
</h1>




In [ ]:
print('12. Parcel Border Points In-Process:')

<h1 style="text-align:right;
           font-size:150%; color:#1c6ce6;">
ספריות

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right;
           font-size:150%; color:#1c6ce6;">
    קריאת נתונים  

In [ ]:
print('  12.a Reading inputs')

In [2]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

<h1 style="text-align:right;font-size:125%">
שכבת נקודת גבול בתהליך

In [ ]:
points_inprocess_cols = ['POINT_NAME', 'ACCURACY', 'SOURCE_CODE', 'MARK_CODE', 'INSERTDATE', 'TALAR_NUM', 'ENTITY_STATE', 'geometry']
points_inprocess = gpd.read_file(RawData_folder + '/CADSDE/Points_InProcess.gdb', layer = 'Points_InProcess', include_fields = points_inprocess_cols)

print('         input 1/2 read')

<h1 style="text-align:right;font-size:125%">
שכבת נקודת בקרה

In [ ]:
control_points = gpd.read_file(RawData_folder + '/CADSDE/CONTROL_POINTS.gdb', layer = 'CONTROL_POINTS', include_fields=['geometry'])

print('         input 2/2 read')

In [ ]:
del [RawData_folder, points_inprocess_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
    עיבוד נתוני נקודות בתהליך  

<h1 style="text-align:right;font-size:100%">
הסרת נקודות שהגיעו מתצ"ר פיקטיבי על ידי מספרי אליפסה לא תקינים
<h1 style="text-align:right;font-size:100%">
הסרת שדה של מספר אליפסה כי אין בו עוד צורך
<h1 style="text-align:right;font-size:100%">
תיקון ערכים של מעמד הנקודה לערכים 1 או 2

In [ ]:
print('  12.b Cleaning data')

In [ ]:
points_inprocess = points_inprocess[points_inprocess['TALAR_NUM'] <= 4000]
points_inprocess.drop(columns='TALAR_NUM', inplace=True)

In [ ]:
points_inprocess['ENTITY_STATE'] = points_inprocess['ENTITY_STATE'] + 1
points_inprocess['ENTITY_STATE'].replace(3, 1, inplace=True)
points_inprocess.fillna(1, inplace=True)

In [ ]:
print('  12.c Data conversion')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
    מציאת נקודות בקרה אשר משמשות גם כנקודות גבול בתהליך
<h1 style="text-align:right;font-size:100%">
    הוספת שדה אינדיקציה לנקודת בקרה עם ערך 1
<h1 style="text-align:right;font-size:100%">
    איחוד מרחבי

In [ ]:
control_points['IsControlBorder'] = 1

points_inprocess = gpd.sjoin(left_df = points_inprocess, right_df = control_points,
                             how = 'left',
                             predicate = 'intersects')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
    מיפוי חדש לערכי סיווג הנקודה
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים ב-0
<h1 style="text-align:right;font-size:100%">
    הגדרת השדה הקיים לשדה מסוג מספרי שלם
<h1 style="text-align:right;font-size:100%">
הגדרת מילון החלפת ערכים בהתאם לאפיון

In [ ]:
points_inprocess['ACCURACY'] = points_inprocess['ACCURACY'].fillna(0)
points_inprocess['ACCURACY'] = points_inprocess['ACCURACY'].astype(int)

class_clf = {0    : 24,
             1    : 1,
             2    : 12,
             3    : 13,
             4    : 13,
             5    : 13,
             6    : 24,
             7    : 24,
             8    : 24,
             9    : 24,
             12   : 12,
             13   : 13,
             24   : 24,
             None : 24}

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
    מיפוי חדש לערכי מקור הנקודה
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים ב-0
<h1 style="text-align:right;font-size:100%">
הגדרת השדה הקיים לשדה מסוג מספרי שלם
<h1 style="text-align:right;font-size:100%">
הגדרת מילון החלפת ערכים בהתאם לאפיון

In [ ]:
points_inprocess['SOURCE_CODE'] = points_inprocess['SOURCE_CODE'].fillna(0)
points_inprocess['SOURCE_CODE'] = points_inprocess['SOURCE_CODE'].astype(int)

source_clf = {0 : 0,
              1 : 0,
              2 : 0,
              3 : 0,
              4 : 0,
              5 : 5,
              6 : 5,
              7 : 5,
              8 : 8,
              9 : 9,
              10 : 10,
              11 : 11,
              12 : 11,
              13 : 13,
              14 : 14,
              15 : 14,
              16 : 16,
              17 : 17,
              18 : 18,
              None : 0
              }

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
    מיפוי חדש לערכי סימון הנקודה
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים ב-0
<h1 style="text-align:right;font-size:100%">
הגדרת השדה הקיים לשדה מסוג מספרי שלם
<h1 style="text-align:right;font-size:100%">
הגדרת מילון החלפת ערכים בהתאם לאפיון

In [ ]:
points_inprocess['MARK_CODE'] = points_inprocess['MARK_CODE'].fillna(0)
points_inprocess['MARK_CODE'] = points_inprocess['MARK_CODE'].astype(int)

mark_clf = {0    : 0,
            1    : 1,
            2    : 2,
            3    : 3,
            4    : 4,
            5    : 5,
            6    : 6,
            7    : 7,
            8    : 8,
            9    : 9,
            10   : 10,
            11   : 11,
            12   : 12,
            13   : 13,
            14   : 14,
            15   : 0,
            16   : 0,
            20   : 20,
            None : 0}

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
    מיפוי חדש לערכי רשת קואורדינטות של הנקודה
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים בתאריך ראשוני
<h1 style="text-align:right;font-size:100%">
הגדרת השדה הקיים לשדה מסוג תאריך
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש ששומר את השנה בלבד
<h1 style="text-align:right;font-size:100%">
מחיקת השדה הקודם
<h1 style="text-align:right;font-size:100%">
הגדרת מילון החלפת ערכים בהתאם לאפיון

In [ ]:
points_inprocess['INSERTDATE'].fillna('01/01/1994', inplace=True)
points_inprocess['INSERTDATE'] = pd.to_datetime(points_inprocess['INSERTDATE'], utc=True)
points_inprocess['geoYear'] = points_inprocess['INSERTDATE'].dt.year
points_inprocess.drop(columns='INSERTDATE', inplace=True)

geodetic_clf = {range(0, 1994)    : 0,
                range(1994, 1999) : 1,
                range(1999, 2009) : 2,
                range(2009, 9999) : 3}

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
   יצירת שכבה חדשה בהתאם לאפיון
<h1 style="text-align:right;font-size:100%">
יצירת טבלה ריקה עם שדות חדשים וסוגיהם
<h1 style="text-align:right;font-size:100%">
איכלס שדות
מחיקת שכבת נקודות גבול בתהליך קודמת לניקוי זיכרון
<h1 style="text-align:right;font-size:100%">
וידוא שהשכבה החדשה מוגדרת ברשת קואורדינטות של ישראל התקפה: 2039 

In [ ]:
print('  12.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("PointUniqueID", int),
                        ("PointName", str),
                        ("IsControlBorder", int),
                        ("Class", int), 
                        ("SourceCode", int),
                        ("MarkCode", int),
                        ("GeodeticNetwork", int),
                        ("Is3D", int),
                        ("Connected3DParcelsIds", int),
                        ("PointStatus", int),
                        ("geometry", gpd.array.GeometryDtype)
                        ])

BorderPoints_InProcess = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                          geometry = 'geometry',
                                          crs = 2039)

In [ ]:
BorderPoints_InProcess['PointUniqueID'] = np.arange(1, len(points_inprocess))
BorderPoints_InProcess['PointUniqueID'] = BorderPoints_InProcess['PointUniqueID'].astype(int)

BorderPoints_InProcess['PointName'] = points_inprocess['POINT_NAME']

BorderPoints_InProcess['IsControlBorder'] = points_inprocess['IsControlBorder']
BorderPoints_InProcess['IsControlBorder'] = BorderPoints_InProcess['IsControlBorder'].fillna(0)
BorderPoints_InProcess['IsControlBorder'] = BorderPoints_InProcess['IsControlBorder'].astype('Int64')

BorderPoints_InProcess['Class'] = points_inprocess['ACCURACY'].map(class_clf)
BorderPoints_InProcess['Class'] = BorderPoints_InProcess['Class'].fillna(24)
BorderPoints_InProcess['Class'] = BorderPoints_InProcess['Class'].astype('Int64')

BorderPoints_InProcess['SourceCode'] = points_inprocess['SOURCE_CODE'].map(source_clf)
BorderPoints_InProcess['SourceCode'] = BorderPoints_InProcess['SourceCode'].fillna(0)
BorderPoints_InProcess['SourceCode'] = BorderPoints_InProcess['SourceCode'].astype('Int64')

BorderPoints_InProcess['MarkCode'] = points_inprocess['MARK_CODE'].map(mark_clf)
BorderPoints_InProcess['MarkCode'] = BorderPoints_InProcess['MarkCode'].fillna(0)
BorderPoints_InProcess['MarkCode'] = points_inprocess['MARK_CODE'].astype('Int64')

BorderPoints_InProcess['GeodeticNetwork'] = points_inprocess['geoYear'].apply(lambda x: next((v for k, v in geodetic_clf.items() if x in k), 0))
BorderPoints_InProcess['GeodeticNetwork'] = BorderPoints_InProcess['GeodeticNetwork'].fillna(0)
BorderPoints_InProcess['GeodeticNetwork'] =BorderPoints_InProcess['GeodeticNetwork'].astype('Int64')

BorderPoints_InProcess['Is3D'] = 0
BorderPoints_InProcess['Is3D'] = BorderPoints_InProcess['Is3D'].astype('Int64')

BorderPoints_InProcess['Connected3DParcelsIds'] = np.nan

BorderPoints_InProcess['PointStatus'] = points_inprocess['ENTITY_STATE']
BorderPoints_InProcess['PointStatus'] = BorderPoints_InProcess['PointStatus'].astype('Int64')

BorderPoints_InProcess['geometry'] = points_inprocess['geometry']


del points_inprocess

In [ ]:
BorderPoints_InProcess.set_geometry('geometry', crs=2039, inplace=True)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות
<h1 style="text-align:right;font-size:100%">
בחירת נתיב
<h1 style="text-align:right;font-size:100%">
שמירה כטבלה       
<h1 style="text-align:right;font-size:100%">
שמירה כשכבה גיאוגרפית

In [ ]:
print('  12.e Exporting results to product folder')

In [ ]:
#csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\BorderPointsInProcess.csv'
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\BorderPointsInProcess.geojson'


#BorderPoints_InProcess.drop(columns='geometry').to_csv(csv_dir ,encoding='utf8', index=False)
BorderPoints_InProcess.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\BorderPoints_InProcess-Save as Feature Class.bat"])